In [1]:
import numpy as np
import itertools
from numba import jit, prange

In [2]:
n_encodings=4
n_ammino=5
n_h=3
n_h_val=2

In [3]:
w=np.zeros((20,3))
v=np.zeros(20)
a=np.zeros(20)
b=np.zeros(3)
h=np.zeros(3)

In [4]:
w.shape

(20, 3)

In [5]:
M=np.zeros((10000,20))

$$E=-\sum_{i,j} v_i w_{i,j} h_j-\sum_i a_i v_i-\sum_j b_j h_j$$
$$p(v,h|a,b,w)=e^{-E(v,h|a,b,w)}/Z$$
$$Z=\sum_{\{v,h\}} e^{-E}$$
$$\mathcal{L}=M^{-1}\sum_{i}^M ln\big[p(v=v_i^{(M)}|a,b,w)\big]$$
$$\mathcal{L}=M^{-1}\left(\sum_i^M{ln\left[\sum_{\{h\}}e^{-E[v_i,h|a,b,w]}\right]}\right) -ln Z$$

In [6]:
def E(v, h, w, a, b):
    return np.exp(np.dot(v, np.dot(w, h)))+np.exp(np.dot(a,v))+np.exp(np.dot(b,h))

In [7]:
def P(v_data, possible_h, w, a, b):
    p=0
    for v_i in v_data:
        sum_h=0
        for h_j in possible_h:
            sum_h+=np.exp(E(v_i, h_j, w, a, b))
        p+=np.log(sum_h)
    return p/v_data.shape[0]

In [8]:
def Z(possible_v, possible_h, w, a, b):
    z=0
    for v_i in possible_v:
        for h_j in possible_h:
            z+=np.exp(E(v_i, h_j, w, a, b))
    return z

In [9]:
def Log(w, a, b):
    return P(v_data, possible_h, w, a, b)-np.log(Z(possible_v, possible_h, w, a, b))

In [10]:
#list all possible combinations of proteins
possibilities_v=list(itertools.product(['1000','0100','0010','0001'], repeat=n_ammino))
possibilities_h=list(itertools.product(['1','0'], repeat=n_h))

possible_v=np.zeros((n_encodings**n_ammino, n_encodings*n_ammino))
possible_h=np.zeros((n_h_val**n_h, n_h))

#converts proteins to decimal representation
for i,v in enumerate(possibilities_v):
    possible_v[i]=np.array([int(i) for i in ''.join(v)])

for i,h in enumerate(possibilities_h):
    possible_h[i]=np.array([int(i) for i in ''.join(h)])

In [11]:
v_data=np.random.randn(10000,20)

In [12]:
Log(w, a, b)

-6.931471805599097